In [1]:
little_bob = 'x18x14'
plain_bob_maj = 'x18x18x18x18'
steadman_trip = '3.1.7.3.1.3.1.3.7.1.3.1'
bristol_max = '-5T-14.5T-5T.36.14-7T.58.16-9T.70.18-18.9T-18-'

In [2]:
def tokenize_place_iter(place):
    current_token = []
    state = 'read'
    for char in place:
        if state == 'read':
            if char == '-' or char == 'x':
                if current_token:
                    state = 'emit_plus'
                else:
                    current_token.append('-')
                    state = 'emit'
            elif char == '.':
                state = 'emit'
            else:
                current_token.append(char)
        if state == 'emit':
            yield ''.join(current_token)
            current_token = []
            state = 'read'
        if state == 'emit_plus':
            yield ''.join(current_token)
            yield '-'
            current_token = []
            state = 'read'
    if current_token:
        yield ''.join(current_token)
        
def tokenize_place(place):
    return list(tokenize_place_iter(place))

In [3]:
assert tokenize_place(little_bob) == ['-', '18', '-', '14']
assert tokenize_place(plain_bob_maj) == ['-', '18', '-', '18', '-', '18', '-', '18']
assert tokenize_place(steadman_trip) == ['3', '1', '7', '3', '1', '3', '1', '3', '7', '1', '3', '1']
assert tokenize_place(bristol_max) == ['-', '5T', '-', '14', '5T', '-', '5T', '36', '14', '-', '7T', '58', '16', '-', '9T', '70', '18', '-', '18', '9T', '-', '18', '-']

In [4]:
import numpy as np

idx = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'E', 'T', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'J', 'K', 'L']
def token_to_indices(token):
    indices = []
    for char in token:
        indices.append(idx.index(char))
    return indices

def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)

def not_indices(full, indices):
    all_indices = np.indices(full.shape)
    return np.setxor1d(all_indices, indices)

bells = np.arange(1, 13)
place = tokenize_place(bristol_max)
print(place)
print(bells)
for _ in range(1):
    for token in place:
        if token == '-':  # swap every pair
            bells = np.ravel(np.flip(np.reshape(bells, (-1, 2)), 1))
        else:
            indices = token_to_indices(token)
            ni = not_indices(bells, indices)
            runs = consecutive(ni)
            for run in runs:
                bells[run] = np.ravel(np.flip(np.reshape(bells[run], (-1, 2)), 1))
        print(bells)
        assert set(bells) == {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

['-', '5T', '-', '14', '5T', '-', '5T', '36', '14', '-', '7T', '58', '16', '-', '9T', '70', '18', '-', '18', '9T', '-', '18', '-']
[ 1  2  3  4  5  6  7  8  9 10 11 12]
[ 2  1  4  3  6  5  8  7 10  9 12 11]
[ 1  2  3  4  6  8  5 10  7 12  9 11]
[ 2  1  4  3  8  6 10  5 12  7 11  9]
[ 2  4  1  3  6  8  5 10  7 12  9 11]
[ 4  2  3  1  6  5  8  7 10  9 12 11]
[ 2  4  1  3  5  6  7  8  9 10 11 12]
[ 4  2  3  1  5  7  6  9  8 11 10 12]
[ 2  4  3  5  1  7  9  6 11  8 12 10]
[ 2  3  4  5  7  1  6  9  8 11 10 12]
[ 3  2  5  4  1  7  9  6 11  8 12 10]
[ 2  3  4  5  7  1  9 11  6 12  8 10]
[ 3  2  5  4  7  9  1 11 12  6 10  8]
[ 3  5  2  7  4  9 11  1  6 12  8 10]
[ 5  3  7  2  9  4  1 11 12  6 10  8]
[ 3  5  2  7  4  9 11  1 12 10  6  8]
[ 5  3  7  2  9  4 11 12  1 10  8  6]
[ 5  7  3  9  2 11  4 12 10  1  6  8]
[ 7  5  9  3 11  2 12  4  1 10  8  6]
[ 7  9  5 11  3 12  2  4 10  1  6  8]
[ 9  7 11  5 12  3  4  2 10  6  1  8]
[ 7  9  5 11  3 12  2  4  6 10  8  1]
[ 7  5  9  3 11  2 12  4 10  6  1